In [ ]:
!pip install py7zr
from glob import glob
import re
import shutil
import pandas as pd
from PIL import Image
import os
from py7zr import unpack_7zarchive
from glob import glob
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
shutil.unpack_archive('../input/cifar-10/train.7z')
shutil.unpack_archive('../input/cifar-10/test.7z')

In [ ]:
train_dir = os.listdir('./train')
train_dir_len = len(train_dir)


In [ ]:

train_labels = pd.read_csv('../input/cifar-10/trainLabels.csv')
train_images = pd.DataFrame(columns=['id', 'label', 'path'], dtype=str)
test_labels = pd.read_csv('../input/cifar-10/sampleSubmission.csv')
train_root = './train/'
for i in range(0, train_dir_len):
    path = path = train_root + str(i+1) + '.png'
    if os.path.exists(path):
        train_images = train_images.append([{
            'id': train_labels['id'].iloc[i], 'label': train_labels['label'].iloc[i], 'path': str(i+1) + '.png' 
        }])
size_img = 32
b_size = 64
quantity_classes = train_images['label'].unique().size
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

for n in classes:
    index = classes.index(n)
    train_images.loc[train_images['label'] == n, 'label'] = str(index)


In [ ]:
data_generator = ImageDataGenerator(rescale=1/255.,
                                   validation_split=0.2,
                                   horizontal_flip=True)

train_generator = data_generator.flow_from_dataframe(dataframe=train_images, 
                                                    directory='./train/',
                                                    x_col='path',
                                                     y_col='label',
                                                     subset='training',
                                                     batch_size=b_size,
                                                     shuffle=True,
                                                     target_size=(size_img,size_img),
                                                     class_mode='categorical')

validation_generator = data_generator.flow_from_dataframe(dataframe=train_images,
                                                         directory='./train/',
                                                         x_col='path',
                                                         y_col='label',
                                                         subset='validation',
                                                         batch_size=b_size,
                                                         shuffle=True,
                                                         target_size=(size_img,size_img),
                                                         class_mode='categorical')

In [ ]:
cnn = models.Sequential()
cnn.add(layers.Conv2D(32, (3, 3), padding='same',
                 input_shape=(size_img, size_img, 3)))
cnn.add(layers.Activation('relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.Conv2D(32, (3, 3)))
cnn.add(layers.Activation('relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPooling2D(pool_size=(2, 2)))
cnn.add(layers.Dropout(0.25))

cnn.add(layers.Conv2D(64, (3, 3), padding='same'))
cnn.add(layers.Activation('relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.Conv2D(64, (3, 3)))
cnn.add(layers.Activation('relu'))
cnn.add(layers.BatchNormalization())
cnn.add(layers.MaxPooling2D(pool_size=(2, 2)))
cnn.add(layers.Dropout(0.25))

cnn.add(layers.Flatten())
cnn.add(layers.Dense(256, kernel_regularizer=l2(0.01)))
cnn.add(layers.Activation('relu'))
cnn.add(layers.Dropout(0.3))
cnn.add(layers.Dense(quantity_classes))
cnn.add(layers.Activation('softmax'))

cnn.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
cnn.summary()

In [ ]:
#history = cnn.fit(train_generator, epochs = 50,  validation_data=validation_generator)
history = cnn.fit(train_generator, 
                    epochs= 50,
                    validation_data=validation_generator,
                    shuffle = True
                   )

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()


plt.plot(history.history['val_accuracy'])
plt.title('model val_accuracy')
plt.ylabel('val_accuracy')
plt.xlabel('epoch')
plt.show()


plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()


plt.plot(history.history['val_loss'])
plt.title('model val_loss')
plt.ylabel('val_loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
!mkdir ./data/
!mv ./test ./data

In [ ]:
test_data_generator = ImageDataGenerator(rescale=1./255.)
test_generator = test_data_generator.flow_from_directory(directory='./data',
            batch_size=b_size,
            shuffle=False,color_mode='rgb',
            target_size=(size_img,size_img),
            class_mode=None)

In [ ]:
prediction = cnn.predict(test_generator)
prediction = np.argmax(prediction, axis=1)
submission = pd.DataFrame(columns=['id', 'label'], dtype=str)
submission['label'] = [classes[int(i)] for i in prediction]
submission['id'] = [ (''.join(filter(str.isdigit, name ))) for name in test_generator.filenames]

submission.sort_values(by=['id'])

In [ ]:
%rm -rf ./train
%rm -rf ./data

In [ ]:
submission.to_csv('my_submission.csv', index=False)